# Kite PnL

In [1]:
import numpy as np
import pandas as pd

In [2]:
df=pd.read_csv("orders.csv")

In [3]:
df.head()

,Time,Type,Instrument,Product,Qty.,Avg. price,Status
0,18/12/21 10:23,SELL,SBI,CNC,1000/1000,525.25,COMPLETE
1,16/12/21 15:08,BUY,ASHOKLEY,MIS,1000/1000,125.70,COMPLETE
2,16/12/21 15:08,BUY,SBI,CNC,1000/1000,520.80,COMPLETE
3,16/12/21 14:13,BUY,TATAMOTORS,MIS,250/250,490.55,COMPLETE
4,16/12/21 13:54,BUY,ASHOKLEY,MIS,0/1000,127.10,CANCELLED


## Data Cleaning

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Time        14 non-null     object 
 1   Type        14 non-null     object 
 2   Instrument  14 non-null     object 
 3   Product     14 non-null     object 
 4   Qty.        14 non-null     object 
 5   Avg. price  14 non-null     float64
 6   Status      14 non-null     object 
dtypes: float64(1), object(6)
memory usage: 916.0+ bytes


In [5]:
df["Qty."]=df["Qty."].str.split("/",expand=True)[0]
df["Qty."]=df["Qty."].astype("int32")

In [6]:
df.head(5)

,Time,Type,Instrument,Product,Qty.,Avg. price,Status
0,18/12/21 10:23,SELL,SBI,CNC,1000,525.25,COMPLETE
1,16/12/21 15:08,BUY,ASHOKLEY,MIS,1000,125.70,COMPLETE
2,16/12/21 15:08,BUY,SBI,CNC,1000,520.80,COMPLETE
3,16/12/21 14:13,BUY,TATAMOTORS,MIS,250,490.55,COMPLETE
4,16/12/21 13:54,BUY,ASHOKLEY,MIS,0,127.10,CANCELLED


In [7]:
df=df[df["Product"]=="MIS"]
df.head()

,Time,Type,Instrument,Product,Qty.,Avg. price,Status
1,16/12/21 15:08,BUY,ASHOKLEY,MIS,1000,125.70,COMPLETE
3,16/12/21 14:13,BUY,TATAMOTORS,MIS,250,490.55,COMPLETE
4,16/12/21 13:54,BUY,ASHOKLEY,MIS,0,127.10,CANCELLED
5,16/12/21 13:21,SELL,TATAMOTORS,MIS,250,492.10,COMPLETE
6,16/12/21 12:51,BUY,TATAMOTORS,MIS,0,490.80,CANCELLED


## Question 1 approach

In [8]:
t1=df.iloc[:,1:8]
t1

,Type,Instrument,Product,Qty.,Avg. price,Status
1,BUY,ASHOKLEY,MIS,1000,125.70,COMPLETE
3,BUY,TATAMOTORS,MIS,250,490.55,COMPLETE
4,BUY,ASHOKLEY,MIS,0,127.10,CANCELLED
5,SELL,TATAMOTORS,MIS,250,492.10,COMPLETE
6,BUY,TATAMOTORS,MIS,0,490.80,CANCELLED
7,SELL,ASHOKLEY,MIS,1000,125.96,COMPLETE
8,BUY,ASHOKLEY,MIS,2000,125.70,COMPLETE
9,SELL,ASHOKLEY,MIS,2000,125.95,COMPLETE
10,SELL,ASHOKLEY,MIS,0,124.45,CANCELLED
11,SELL,ASHOKLEY,MIS,2000,125.95,COMPLETE


In [9]:
def brokerage(num):
    if num*(0.03/100)<20:
        return num*(0.03/100)
    else:
        return 20

def stt_ctt(df):
    if df["Type"]=="SELL":
        return df["Turnover"]*(0.025/100)
    else:
        return 0

def stampcharges(df):
    if df["Type"]=="BUY":
        return df["Turnover"]*(0.003/100)
    else:
        return 0

In [10]:
t1["Turnover"]=t1["Avg. price"]*t1["Qty."]
t1["Brokerage"]=t1["Turnover"].apply(brokerage)
t1["STT/CTT"]=t1.apply(stt_ctt,axis=1)
t1["ETC"]=t1["Turnover"]*(0.00322/100)
t1["SEBI"]=t1["Turnover"]*(10/10**7)
t1["GST"]=(t1["Brokerage"]+t1["SEBI"]+t1["ETC"])*(18/100)
t1["Stamp Charges"]=t1.apply(stampcharges,axis=1)
t1["Total Charges"]=t1["Brokerage"]+t1["STT/CTT"]+t1["ETC"]+t1["SEBI"]+t1["GST"]+t1["Stamp Charges"]
t1.rename({"Instrument":"Stock"},axis=1,inplace=True)
# t1.drop(["number_of_shares_executed"],axis=1,inplace=True)
t1

,Type,Stock,Product,Qty.,Avg. price,Status,Turnover,Brokerage,STT/CTT,ETC,SEBI,GST,Stamp Charges,Total Charges
1,BUY,ASHOKLEY,MIS,1000,125.70,COMPLETE,125700.0,20.0,0.00000,4.047540,0.125700,4.351183,3.771000,32.295423
3,BUY,TATAMOTORS,MIS,250,490.55,COMPLETE,122637.5,20.0,0.00000,3.948928,0.122637,4.332882,3.679125,32.083572
4,BUY,ASHOKLEY,MIS,0,127.10,CANCELLED,0.0,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
5,SELL,TATAMOTORS,MIS,250,492.10,COMPLETE,123025.0,20.0,30.75625,3.961405,0.123025,4.335197,0.000000,59.175877
6,BUY,TATAMOTORS,MIS,0,490.80,CANCELLED,0.0,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
7,SELL,ASHOKLEY,MIS,1000,125.96,COMPLETE,125960.0,20.0,31.49000,4.055912,0.125960,4.352737,0.000000,60.024609
8,BUY,ASHOKLEY,MIS,2000,125.70,COMPLETE,251400.0,20.0,0.00000,8.095080,0.251400,5.102366,7.542000,40.990846
9,SELL,ASHOKLEY,MIS,2000,125.95,COMPLETE,251900.0,20.0,62.97500,8.111180,0.251900,5.105354,0.000000,96.443434
10,SELL,ASHOKLEY,MIS,0,124.45,CANCELLED,0.0,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
11,SELL,ASHOKLEY,MIS,2000,125.95,COMPLETE,251900.0,20.0,62.97500,8.111180,0.251900,5.105354,0.000000,96.443434


## Question 2 approach

In [11]:
t2=t1.drop(["Product","Status"],axis=1)
t2

,Type,Stock,Qty.,Avg. price,Turnover,Brokerage,STT/CTT,ETC,SEBI,GST,Stamp Charges,Total Charges
1,BUY,ASHOKLEY,1000,125.70,125700.0,20.0,0.00000,4.047540,0.125700,4.351183,3.771000,32.295423
3,BUY,TATAMOTORS,250,490.55,122637.5,20.0,0.00000,3.948928,0.122637,4.332882,3.679125,32.083572
4,BUY,ASHOKLEY,0,127.10,0.0,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
5,SELL,TATAMOTORS,250,492.10,123025.0,20.0,30.75625,3.961405,0.123025,4.335197,0.000000,59.175877
6,BUY,TATAMOTORS,0,490.80,0.0,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
7,SELL,ASHOKLEY,1000,125.96,125960.0,20.0,31.49000,4.055912,0.125960,4.352737,0.000000,60.024609
8,BUY,ASHOKLEY,2000,125.70,251400.0,20.0,0.00000,8.095080,0.251400,5.102366,7.542000,40.990846
9,SELL,ASHOKLEY,2000,125.95,251900.0,20.0,62.97500,8.111180,0.251900,5.105354,0.000000,96.443434
10,SELL,ASHOKLEY,0,124.45,0.0,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
11,SELL,ASHOKLEY,2000,125.95,251900.0,20.0,62.97500,8.111180,0.251900,5.105354,0.000000,96.443434


In [12]:
dft2=t2.groupby(["Stock","Type"])[["Qty.","Avg. price","Turnover","Brokerage","STT/CTT","ETC","SEBI","GST","Stamp Charges","Total Charges"]].agg({"Qty.":"sum","Avg. price":"mean","Turnover":"sum","Brokerage":"sum","STT/CTT":"sum","ETC":"sum","SEBI":"sum","GST":"sum","Stamp Charges":"sum","Total Charges":"sum"})
dft2

Qty.  Avg. price  Turnover  Brokerage    STT/CTT        ETC  \
Stock      Type                                                                
ASHOKLEY   BUY   5000    125.9500  628350.0       80.0    0.00000  20.232870   
           SELL  5000    125.5775  629760.0       60.0  157.44000  20.278272   
TATAMOTORS BUY    250    490.6750  122637.5       20.0    0.00000   3.948928   
           SELL   250    492.1000  123025.0       20.0   30.75625   3.961405   

                     SEBI        GST  Stamp Charges  Total Charges  
Stock      Type                                                     
ASHOKLEY   BUY   0.628350  18.155020      18.850500     137.866740  
           SELL  0.629760  14.563446       0.000000     252.911478  
TATAMOTORS BUY   0.122637   4.332882       3.679125      32.083572  
           SELL  0.123025   4.335197       0.000000      59.175877

## Question 3 approach

In [13]:
t3=pd.DataFrame()
t3["Instrument"]=t2["Stock"].unique()
t3

,Instrument
0,ASHOKLEY
1,TATAMOTORS


In [14]:
t3=pd.DataFrame(columns=["Instrument","BUY","SELL"])
t3["Instrument"]=t2["Stock"].unique()
print(t3)

for i in range(len(t2['Stock'].unique())):
    
    for j in range(len(t2["Type"].unique())) :
        t3[f"{t2['Type'].unique()[j]}"].iloc[i]=t2[(t2["Type"]==f"{t2['Type'].unique()[j]}") & (t2["Stock"]==f"{t2['Stock'].unique()[i]}")]["Turnover"].sum()
t3

   Instrument  BUY SELL
0    ASHOKLEY  NaN  NaN
1  TATAMOTORS  NaN  NaN


,Instrument,BUY,SELL
0,ASHOKLEY,628350.0,629760.0
1,TATAMOTORS,122637.5,123025.0


In [15]:
t3["Gross PnL"]=t3["SELL"]-t3["BUY"]
t3["Total Charges"]=t2.groupby("Stock")["Total Charges"].sum().reset_index()["Total Charges"]
t3["Net PnL"]=t3["Gross PnL"]-t3["Total Charges"]
t3["% Charges on Gross PnL"]=(t3["Total Charges"]/t3["Gross PnL"])*100
t3

,Instrument,BUY,SELL,Gross PnL,Total Charges,Net PnL,% Charges on Gross PnL
0,ASHOKLEY,628350.0,629760.0,1410.0,390.778217,1019.221783,27.714767
1,TATAMOTORS,122637.5,123025.0,387.5,91.259449,296.240551,23.550826


In [16]:
with pd.ExcelWriter("kite.xlsx", engine="openpyxl") as writer:
    t1.to_excel(writer, sheet_name="Question1")
    dft2.to_excel(writer, sheet_name="Question2")
    t3.to_excel(writer, sheet_name="Question3")